# Module 8: Concrete Security

## Latice Estimator

### LWE

#### Basics

In [1]:
from estimator import *
from estimator.nd import NoiseDistribution, stddevf
from estimator.lwe_parameters import LWEParameters
from estimator.lwe import estimate

In [2]:
MyParam = LWEParameters(n=256, q=1024, 
                Xs=NoiseDistribution.CenteredBinomial(3),
                Xe=NoiseDistribution.SparseTernary(256, 40, 40),
                m=256, 
                tag="MyParam",
                )
r = LWE.estimate.rough(MyParam)		# usvp & dual_hybrid only

usvp                 :: rop: ≈2^52.0, red: ≈2^52.0, δ: 1.006744, β: 178, d: 466, tag: usvp
dual_hybrid          :: rop: ≈2^51.8, red: ≈2^51.7, guess: ≈2^48.3, β: 177, p: 3, ζ: 0, t: 20, β': 177, N: ≈2^36.5, m: 256


In [3]:
r = LWE.estimate(MyParam)		# more attacks

arora-gb             :: rop: ≈2^434.4, dreg: 54, mem: ≈2^272.5, t: 3, m: ≈2^169.9, tag: arora-gb, ↻: ≈2^161.9, ζ: 162, |S|: ≈2^92.3, prop: ≈2^-67.4
bkw                  :: rop: ≈2^83.2, m: ≈2^72.6, mem: ≈2^73.6, b: 7, t1: 0, t2: 12, ℓ: 6, #cod: 212, #top: 0, #test: 44, tag: coded-bkw
usvp                 :: rop: ≈2^79.7, red: ≈2^79.7, δ: 1.006744, β: 178, d: 466, tag: usvp
bdd                  :: rop: ≈2^76.9, red: ≈2^75.8, svp: ≈2^75.9, β: 164, η: 194, d: 442, tag: bdd
dual                 :: rop: ≈2^82.7, mem: ≈2^48.9, m: 234, β: 185, d: 490, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^76.3, red: ≈2^76.2, guess: ≈2^71.8, β: 161, p: 3, ζ: 15, t: 20, β': 177, N: ≈2^36.3, m: 256


<b>Remark.</b>
Beta prime (β') is (possibly) reduced BKZ block size due to "Dimension for free" technique: Léo Ducas, Shortest Vector from Lattice Sieving: a Few Dimensions for Free, Eurocrypt 2018. 

#### Cost Models

In [4]:
from estimator.reduction import *

In [5]:
r = LWE.estimate(MyParam, red_cost_model=ADPS16("classical"))

arora-gb             :: rop: ≈2^434.4, dreg: 54, mem: ≈2^272.5, t: 3, m: ≈2^169.9, tag: arora-gb, ↻: ≈2^161.9, ζ: 162, |S|: ≈2^92.3, prop: ≈2^-67.4
bkw                  :: rop: ≈2^83.2, m: ≈2^72.6, mem: ≈2^73.6, b: 7, t1: 0, t2: 12, ℓ: 6, #cod: 212, #top: 0, #test: 44, tag: coded-bkw
usvp                 :: rop: ≈2^52.0, red: ≈2^52.0, δ: 1.006744, β: 178, d: 466, tag: usvp
bdd                  :: rop: ≈2^53.0, red: ≈2^52.0, svp: ≈2^52.0, β: 178, η: 178, d: 467, tag: bdd
dual                 :: rop: ≈2^56.1, mem: ≈2^47.4, m: 240, β: 192, d: 496, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^51.8, red: ≈2^51.7, guess: ≈2^48.3, β: 177, p: 3, ζ: 0, t: 20, β': 177, N: ≈2^36.5, m: 256


In [6]:
# Check Core-SVP
print("usvp        :", numerical_approx(0.292*178, digits=3))
print("bdd         :", numerical_approx(0.292*178, digits=3))
print("dual        :", numerical_approx(0.292*192, digits=3))
print("dual_hybrid :", numerical_approx(0.292*177, digits=3))

usvp        : 52.0
bdd         : 52.0
dual        : 56.1
dual_hybrid : 51.7


In [7]:
r = LWE.estimate(MyParam, red_cost_model=ADPS16("quantum"))

arora-gb             :: rop: ≈2^434.4, dreg: 54, mem: ≈2^272.5, t: 3, m: ≈2^169.9, tag: arora-gb, ↻: ≈2^161.9, ζ: 162, |S|: ≈2^92.3, prop: ≈2^-67.4
bkw                  :: rop: ≈2^83.2, m: ≈2^72.6, mem: ≈2^73.6, b: 7, t1: 0, t2: 12, ℓ: 6, #cod: 212, #top: 0, #test: 44, tag: coded-bkw
usvp                 :: rop: ≈2^47.2, red: ≈2^47.2, δ: 1.006744, β: 178, d: 466, tag: usvp
bdd                  :: rop: ≈2^48.2, red: ≈2^47.2, svp: ≈2^47.2, β: 178, η: 178, d: 467, tag: bdd
dual                 :: rop: ≈2^50.9, mem: ≈2^47.4, m: 240, β: 192, d: 496, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^48.3, red: ≈2^47.7, guess: ≈2^46.7, β: 180, p: 2, ζ: 0, t: 25, β': 180, N: ≈2^37.0, m: 256


In [8]:
r = LWE.estimate(MyParam, red_cost_model=ChaLoy21)

Algorithm functools.partial(<function dual at 0x7eb3f29885e0>, red_cost_model=<class 'estimator.reduction.ChaLoy21'>) on LWEParameters(n=256, q=1024, Xs=D(σ=0.56), Xe=D(σ=1.22), m=256, tag='MyParam') failed with _short_vectors_sieve() missing 1 required positional argument: 'self'
Algorithm functools.partial(<estimator.lwe_dual.MATZOV object at 0x7eb3f298bf40>, red_cost_model=<class 'estimator.reduction.ChaLoy21'>) on LWEParameters(n=256, q=1024, Xs=D(σ=0.56), Xe=D(σ=1.22), m=256, tag='MyParam') failed with _short_vectors_sieve() missing 1 required positional argument: 'beta'
arora-gb             :: rop: ≈2^434.4, dreg: 54, mem: ≈2^272.5, t: 3, m: ≈2^169.9, tag: arora-gb, ↻: ≈2^161.9, ζ: 162, |S|: ≈2^92.3, prop: ≈2^-67.4
bkw                  :: rop: ≈2^83.2, m: ≈2^72.6, mem: ≈2^73.6, b: 7, t1: 0, t2: 12, ℓ: 6, #cod: 212, #top: 0, #test: 44, tag: coded-bkw
usvp                 :: rop: ≈2^45.7, red: ≈2^45.7, δ: 1.006744, β: 178, d: 466, tag: usvp
bdd                  :: rop: ≈2^46.7, red

### SIS

In [9]:
params = SIS.Parameters(n=113, q=2048, length_bound=512, norm=2)
SIS.lattice(params)

rop: ≈2^47.0, red: ≈2^47.0, δ: 1.011391, β: 61, d: 276, tag: euclidean

In [10]:
params = SIS.Parameters(n=113, q=2048, length_bound=50, norm=oo)
SIS.lattice(params)

rop: ≈2^52.2, red: ≈2^51.1, sieve: ≈2^51.3, β: 76, η: 96, ζ: 101, d: 273, prob: 1, ↻: 1, tag: infinity

### NTRU

In [11]:
params = NTRU.Parameters(n=200, q=7981, Xs=ND.UniformMod(3), Xe=ND.UniformMod(3))
NTRU.primal_usvp(params, red_shape_model="gsa")
NTRU.primal_usvp(params, red_shape_model=Simulator.CN11)

rop: ≈2^52.8, red: ≈2^52.8, δ: 1.010263, β: 80, d: 385, tag: usvp

In [12]:
NTRU.primal_hybrid(params, red_shape_model=Simulator.CN11)

rop: ≈2^67.3, red: ≈2^66.6, svp: ≈2^66.0, β: 79, η: 2, ζ: 44, |S|: ≈2^69.7, d: 356, prob: ≈2^-12.0, ↻: ≈2^14.2, tag: hybrid

In [13]:
params.possibly_overstretched
NTRU.primal_dsd(params, red_shape_model=Simulator.ZGSA) 

rop: ≈2^42.6, red: ≈2^42.6, δ: 1.012427, β: 43, d: 400, tag: dsd

In [14]:
NTRU.primal_dsd(params, red_shape_model=Simulator.CN11)

rop: ≈2^42.2, red: ≈2^42.2, δ: 1.012468, β: 42, d: 400, tag: dsd

## Kyber

### Parameters

In [15]:
n = 256
q = 3329

In [16]:
# Kyber512
tag = "Kyber512"
k = 2
eta1 = 3
eta2 = 3
eta3 = 2 # ctxt error distribution for level 1, otherwise eta2=eta3
d1 = 10
d2 = 4

In [17]:
# Kyber768
tag = "Kyber768"
k = 3
eta1 = 2
eta2 = 2
eta3 = 2
d1 = 10
d2 = 4

In [18]:
# Kyber1024
tag = "Kyber1024"
k = 4
eta1 = 2
eta2 = 2
eta3 = 2
d1 = 11
d2 = 5 

### Security Estimation

In [19]:
Kyber = LWEParameters(n=n*k, q=q, 
                	Xs=NoiseDistribution.CenteredBinomial(eta1),
                	Xe=NoiseDistribution.CenteredBinomial(eta2),
                	m=n*(k+1), tag=tag)

In [20]:
print("*"+tag+"*")
r = LWE.estimate(Kyber, red_cost_model=ADPS16("classical"))

*Kyber1024*
bkw                  :: rop: ≈2^310.5, m: ≈2^296.8, mem: ≈2^296.7, b: 25, t1: 0, t2: 18, ℓ: 24, #cod: 897, #top: 0, #test: 129, tag: coded-bkw
usvp                 :: rop: ≈2^255.2, red: ≈2^255.2, δ: 1.002262, β: 874, d: 1867, tag: usvp
bdd                  :: rop: ≈2^256.1, red: ≈2^254.9, svp: ≈2^255.2, β: 873, η: 874, d: 1885, tag: bdd
dual                 :: rop: ≈2^264.0, mem: ≈2^197.4, m: 918, β: 904, d: 1942, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^241.8, red: ≈2^241.2, guess: ≈2^240.1, β: 826, p: 4, ζ: 0, t: 110, β': 826, N: ≈2^171.4, m: 1024


### Decryption Failures

In [21]:
from crystals import *
import operator as op
from math import factorial as fac
from math import sqrt, log
import sys
from crystals.proba_util import *

/home/hyeongmin/PQC_training/module8/crystals/proba_util.py:6: DeprecationWarning: invalid escape sequence \s
  """ Weight of the gaussian of std deviation s, on the interval [-t, t]


In [22]:
# MLWE secrets in pk (s) and cttxt (r)   : CBD(eta1)
# MLWE errors in pk (e) and ctxt (e1, e2): CBD(eta2), CBD(eta2), CBD(eta3)
# Initialize secrets and errors
Ds = build_centered_binomial_law(eta1)      # equal to Dr
De = build_centered_binomial_law(eta2)      # equal to De1, De2
De1 = build_centered_binomial_law(eta3)     # equal to De2

# Compression errors in ctxt (c1, c2), 
# each are ModSwitch errors from q to 2^d1 and 2^d2
Dc1 = build_mod_switching_error_law(q, 2**d1)
Dc2 = build_mod_switching_error_law(q, 2**d2)

# Combinations
# e*r + e2 + c2 - s*e1 - s*c1
# We ignore the publlic key compression noise, which is small
Dc1_e1 = law_convolution(Dc1, De1)                              # c1+e1
Der = iter_law_convolution(law_product(De, Ds), n*k)            # er
Dse1_sc1 = iter_law_convolution(law_product(Ds, Dc1_e1), n*k)   # se1+sc1

D = law_convolution(Der, Dse1_sc1)      # er-se1-sc1
D = law_convolution(D, De1)             # er+e2-se1-sc1
D = law_convolution(D, Dc2)             # final

prob = tail_probability(D, q/4)

print("*"+tag+"*")
print("DFP:", log(n*prob)/log(2))

*Kyber1024*
DFP: -175.19606603970226


## Smaug

### Parameters

In [23]:
n = 256
sigma = 1.0625

In [24]:
# Smaug512
tag = "Smaug512"
k = 2
q = 1024
p = 256
pp = 32
h = 140
numCBD = 2

In [25]:
# Smaug768
tag = "Smaug768"
k = 3
q = 2048
p = 512
pp = 16
h = 264
numCBD = 4

In [26]:
# Smaug512
tag = "Smaug512"
k = 4
q = 2048
p = 512
pp = 128
h = 348
numCBD = 3

### Security Estimate

Define 'ModifiedCBD' in “estimator/nd.py”:

    @staticmethod
    def ModifiedCBD(numCBD, n=None):
        #if numCBD == 1:
        #    # -1: 1/16, 0: 14/16, 1: 1/16
        if numCBD == 2:
            # -1: 1/8, 0: 3/4, 1: 1/8
            # mean = 0, stddev**2 = 1/8*1*2 = 1/4
            D = NoiseDistribution(n=n, stddev=RR(1/2), mean=RR(0), density=1/RR(4), bounds=(-1, 1), tag="modifiedCBD")
        elif numCBD == 3:
            # -1: 3/16, 0: 5/8, 1: 3/16
            # mean = 0, stddev**2 = 3/16*1*2 = 3/8
            D = NoiseDistribution(n=n, stddev=sqrt(3/RR(8)), mean=RR(0), density=3/RR(8), bounds=(-1, 1), tag="modifiedCBD")
        elif numCBD == 4:
            D = NoiseDistribution.CenteredBinomial(1)
        return D

In [27]:
Smaug_LWE = LWEParameters(n=n*k, q=q, 
                	Xs=NoiseDistribution.SparseTernary(n*k, h/2, h/2),
                	Xe=NoiseDistribution.DiscreteGaussian(sigma),
                	m=n*k, tag=tag+"LWE")

Smaug_LWR = LWEParameters(n=n*k, q=q, 
                	Xs=NoiseDistribution.ModifiedCBD(numCBD),
                	Xe=NoiseDistribution.UniformMod(q/p),
                	m=n*(k+1), tag=tag+"LWR")

In [28]:
print("*"+tag+"*")
r = LWE.estimate(Smaug_LWE, red_cost_model=ADPS16("classical"))

*Smaug512*
arora-gb             :: rop: ≈2^inf, m: ≈2^752.1, dreg: 268, t: 4, mem: ≈2^inf, tag: arora-gb, ↻: ≈2^746.3, ζ: 677, |S|: ≈2^562.1, prop: ≈2^-182.0
bkw                  :: rop: ≈2^269.0, m: ≈2^256.0, mem: ≈2^257.0, b: 23, t1: 0, t2: 16, ℓ: 22, #cod: 863, #top: 0, #test: 163, tag: coded-bkw
usvp                 :: rop: ≈2^251.1, red: ≈2^251.1, δ: 1.002289, β: 860, d: 1749, tag: usvp
bdd                  :: rop: ≈2^252.3, red: ≈2^251.4, svp: ≈2^251.1, β: 861, η: 860, d: 1731, tag: bdd
dual                 :: rop: ≈2^260.5, mem: ≈2^194.9, m: 775, β: 892, d: 1799, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^231.5, red: ≈2^231.3, guess: ≈2^228.7, β: 792, p: 2, ζ: 25, t: 170, β': 792, N: ≈2^164.3, m: 1024


In [29]:
print("*"+tag+"*")
r = LWE.estimate(Smaug_LWR, red_cost_model=ADPS16("classical"))

*Smaug512*
Algorithm <estimator.lwe_bkw.CodedBKW object at 0x7eb3f2bc8d90> on LWEParameters(n=1024, q=2048, Xs=D(σ=0.61), Xe=D(σ=1.12, μ=-0.50), m=1280, tag='Smaug512LWR') failed with Amplifying for μ≠0 not implemented.
arora-gb             :: rop: ≈2^inf, dreg: 158, mem: ≈2^inf, t: 2, m: ≈2^528.6, tag: arora-gb, ↻: ≈2^518.3, ζ: 491, |S|: ≈2^184.3, prop: ≈2^-331.8
usvp                 :: rop: ≈2^254.9, red: ≈2^254.9, δ: 1.002264, β: 873, d: 1731, tag: usvp
bdd                  :: rop: ≈2^255.8, red: ≈2^254.9, svp: ≈2^254.6, β: 873, η: 872, d: 1751, tag: bdd
dual                 :: rop: ≈2^264.3, mem: ≈2^197.6, m: 784, β: 905, d: 1808, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^238.3, red: ≈2^238.3, guess: ≈2^232.4, β: 816, p: 2, ζ: 5, t: 205, β': 816, N: ≈2^168.6, m: 1024


### Decryption Failures

Define 'build_mCBD_law' in “crystals/proba_util.py”:

    def build_mCBD_law(numCBD):
        D = {}
        D[-1] = numCBD/16.0
        D[1] = numCBD/16.0
        D[0] = 1.0-D[1]-D[-1]
        return D

In [30]:
##############
# initialize #
##############
# LWE error (dGaussian, sigma=1.0625)
D_e = {-3: 7.0/1024, -2: 65.0/1024, -1: 247.0/1024, 0:384.0/1024, 1: 247.0/1024, 2: 65.0/1024, 3: 7.0/1024}

# LWR secret (modifiedCBD)
D_r = build_mCBD_law(numCBD)

# LWR error for ctxt    (ModSwitch q->p and pp)
D_e1 = build_mod_switching_error_law(q, p)
D_e2 = build_mod_switching_error_law(q, pp)

###########
# combine #
###########
D_er = iter_law_convolution(law_product(D_e, D_r), k*n)    # <e, r>
D_e1s = iter_law_convolution(D_e1, h)                      # <e1, s> = e1+...+e1, hs times.

D_er_e1s = law_convolution(D_er, D_e1s)            # <e, r> + <e1, s>
# convolution_remove_dependency(D_er, D_e1s, q, p) # <e, r> + <e1, s>

D = law_convolution(D_er_e1s, D_e2)                # <e, r> + <e1, s> + e2

prob = tail_probability(D, q/4.0)                  # Pr[ |<e, r> + <e1, s> + e2| > q/4 ]

if prob!=0:
    prob = log(n*prob,2)                           # for each n coefficients

print("*"+tag+"*")
print(prob)

*Smaug512*
-194.1604380337613


## Falcon

### Parameters

In [31]:
q = 12289.0

In [32]:
# Falcon512
tag = "Falcon512"
n = 512.0
betaSquared=34034726
Ran_KR = range(450, 460)
Ran_Unf = range(405, 415)

In [33]:
# Falcon1024
tag = "Falcon1024"
n = 1024.0
betaSquared=70265242
Ran_KR = range(930, 940)
Ran_Unf = range(945, 955)

### Key Recovery

In [34]:
# Falcon_Key_Recovery
print("*"+tag+"*")
for B in Ran_KR:
    # Attack succeed when Left > Right
    # with attack costs 2^(0.292*B) [Core-SVP]
    left = numerical_approx((B/2/pi/e)^(1-n/B)*sqrt(q))
    right = numerical_approx(1.17*sqrt(3/4*B*q/2/n))
    suc = (left>right)
    print("* B="+str(B)+":: Left:", left, "   //   Right:", right, "\n** Left", ('>' if suc else '<'), "Right:: ", ("Attack cost: "+str(0.292*B) if suc else "Attack failed!"))

*Falcon1024*
* B=930:: Left: 74.0100566051179    //   Right: 75.6922236079881 
** Left < Right::  Attack failed!
* B=931:: Left: 74.3527877254226    //   Right: 75.7329074182936 
** Left < Right::  Attack failed!
* B=932:: Left: 74.6965461770597    //   Right: 75.7735693849378 
** Left < Right::  Attack failed!
* B=933:: Left: 75.0413330571217    //   Right: 75.8142095430674 
** Left < Right::  Attack failed!
* B=934:: Left: 75.3871494598636    //   Right: 75.8548279277350 
** Left < Right::  Attack failed!
* B=935:: Left: 75.7339964767047    //   Right: 75.8954245738993 
** Left < Right::  Attack failed!
* B=936:: Left: 76.0818751962305    //   Right: 75.9359995164256 
** Left > Right::  Attack cost: 273.312000000000
* B=937:: Left: 76.4307867041942    //   Right: 75.9765527900861 
** Left > Right::  Attack cost: 273.604000000000
* B=938:: Left: 76.7807320835185    //   Right: 76.0170844295603 
** Left > Right::  Attack cost: 273.896000000000
* B=939:: Left: 77.1317124142972    //   R

### Signature Forgery

In [39]:
# Falcon_Signature_Forgery
print("*"+tag+"*")
for B in Ran_Unf:
    # Attack succeed when Left < Right
    # with attack costs 2^(0.292*B) [Core-SVP]
    left = numerical_approx((B/2/pi/e)^(n/B)*sqrt(q))
    right = numerical_approx(sqrt(betaSquared))
    suc = (left<right)
    print("* B="+str(B)+":: Left:", left, "   //   Right:", right, "\n** Left", ('<' if suc else '>'), "Right:: ", ("Attack cost: "+str(0.292*B) if suc else "Attack failed!"))

*Falcon1024*
* B=945:: Left: 8578.73564276405    //   Right: 8382.43651929438 
** Left > Right::  Attack failed!
* B=946:: Left: 8549.17114018353    //   Right: 8382.43651929438 
** Left > Right::  Attack failed!
* B=947:: Left: 8519.76034605765    //   Right: 8382.43651929438 
** Left > Right::  Attack failed!
* B=948:: Left: 8490.50223263106    //   Right: 8382.43651929438 
** Left > Right::  Attack failed!
* B=949:: Left: 8461.39578044934    //   Right: 8382.43651929438 
** Left > Right::  Attack failed!
* B=950:: Left: 8432.43997828092    //   Right: 8382.43651929438 
** Left > Right::  Attack failed!
* B=951:: Left: 8403.63382303983    //   Right: 8382.43651929438 
** Left > Right::  Attack failed!
* B=952:: Left: 8374.97631970925    //   Right: 8382.43651929438 
** Left < Right::  Attack cost: 277.984000000000
* B=953:: Left: 8346.46648126588    //   Right: 8382.43651929438 
** Left < Right::  Attack cost: 278.276000000000
* B=954:: Left: 8318.10332860517    //   Right: 8382.4365

### Key Recovery (lattice estimator)

In [35]:
schemes.Falcon512_SKR

NTRUParameters(n=512, q=12289, Xs=D(σ=4.05), Xe=D(σ=4.05), m=512, tag='Falcon512_SKR', ntru_type='circulant')

In [36]:
r = NTRU.estimate(schemes.Falcon512_SKR, red_cost_model=ADPS16("classical"))

usvp                 :: rop: ≈2^141.0, red: ≈2^141.0, δ: 1.003489, β: 483, d: 1020, tag: usvp
bdd                  :: rop: ≈2^141.6, red: ≈2^140.5, svp: ≈2^140.7, β: 481, η: 482, d: 1022, tag: bdd
bdd_hybrid           :: rop: ≈2^141.5, red: ≈2^140.5, svp: ≈2^140.5, β: 481, η: 481, ζ: 0, |S|: 1, d: 1024, prob: 1, ↻: 1, tag: hybrid
bdd_mitm_hybrid      :: rop: ≈2^325.2, red: ≈2^325.2, svp: ≈2^204.8, β: 481, η: 2, ζ: 0, |S|: 1, d: 1024, prob: ≈2^-182.6, ↻: ≈2^184.8, tag: hybrid


In [37]:
schemes.Falcon1024_SKR

NTRUParameters(n=1024, q=12289, Xs=D(σ=2.87), Xe=D(σ=2.87), m=1024, tag='Falcon1024_SKR', ntru_type='circulant')

In [38]:
r = NTRU.estimate(schemes.Falcon1024_SKR, red_cost_model=ADPS16("classical"))

usvp                 :: rop: ≈2^284.4, red: ≈2^284.4, δ: 1.002084, β: 974, d: 2047, tag: usvp
bdd                  :: rop: ≈2^285.1, red: ≈2^284.1, svp: ≈2^284.1, β: 973, η: 973, d: 2046, tag: bdd
bdd_hybrid           :: rop: ≈2^285.1, red: ≈2^284.1, svp: ≈2^284.1, β: 973, η: 973, ζ: 0, |S|: 1, d: 2048, prob: 1, ↻: 1, tag: hybrid
bdd_mitm_hybrid      :: rop: ≈2^756.3, red: ≈2^756.3, svp: ≈2^494.2, β: 973, η: 2, ζ: 0, |S|: 1, d: 2048, prob: ≈2^-470.0, ↻: ≈2^472.2, tag: hybrid


### Signature Forgery (lattice estimator)

In [40]:
schemes.Falcon512_Unf

SISParameters(n=512, q=12289, length_bound=5833.9072, m=1024, norm=2, tag='Falcon512_Unf')

In [41]:
r = SIS.estimate(schemes.Falcon512_Unf, red_cost_model=ADPS16("classical"))

lattice  :: rop: ≈2^121.2, red: ≈2^121.2, δ: 1.003882, β: 415, d: 1024, tag: euclidean


In [42]:
schemes.Falcon1024_Unf

SISParameters(n=1024, q=12289, length_bound=8382.4081, m=2048, norm=2, tag='Falcon1024_Unf')

In [43]:
r = SIS.estimate(schemes.Falcon1024_Unf, red_cost_model=ADPS16("classical"))

lattice  :: rop: ≈2^279.2, red: ≈2^279.2, δ: 1.002114, β: 956, d: 2048, tag: euclidean


## Dilithium

### Parameters

In [44]:
n = 256
q = 8380417
d = 13

In [45]:
# Dilithium2
tag = "Dilithium2"
k = 4
l = 4
eta = 4
tau = 39
gamma1 = 2**17
gamma2 = (q-1)/88

In [62]:
# Dilithium3
tag = "Dilithium3"
k = 6
l = 5
eta = 2
tau = 49
gamma1 = 2**19
gamma2 = (q-1)/32

In [47]:
# Dilithium5
tag = "Dilithium5"
k = 8
l = 7
eta = 2
tau = 60
gamma1 = 2**19
gamma2 = (q-1)/32

### Key Recovery

In [63]:
Dilithium_KR = LWEParameters(n=n*k, q=q, 
                	Xs=NoiseDistribution.Uniform(-eta,eta),
                	Xe=NoiseDistribution.Uniform(-eta,eta),
                	m=n*l, tag=tag)

print("*"+tag+"*")
r=LWE.estimate(Dilithium_KR, red_cost_model=ADPS16("classical"))

*Dilithium3*
bkw                  :: rop: ≈2^386.8, m: ≈2^372.2, mem: ≈2^373.2, b: 16, t1: 0, t2: 37, ℓ: 15, #cod: 1420, #top: 0, #test: 123, tag: coded-bkw
usvp                 :: rop: ≈2^209.9, red: ≈2^209.9, δ: 1.002615, β: 719, d: 2810, tag: usvp
bdd                  :: rop: ≈2^210.8, red: ≈2^209.9, svp: ≈2^209.7, β: 719, η: 718, d: 2816, tag: bdd
dual                 :: rop: ≈2^214.6, mem: ≈2^162.0, m: 1280, β: 735, d: 2816, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^203.1, red: ≈2^202.9, guess: ≈2^199.7, β: 695, p: 5, ζ: 10, t: 65, β': 695, N: ≈2^143.7, m: 1536


### Signature Forgery

In [64]:
beta = tau*eta
zeta = max(gamma1-beta, 2*gamma2 + 1 + 2**(d-1)*tau)
zetap = max(2*(gamma1-beta), 4*gamma2+2)

print("*"+tag+"*")
print("SelfTargetMSIS bound:", zeta)
print("MSIS bound:", zetap)

*Dilithium3*
SelfTargetMSIS bound: 724481
MSIS bound: 1048380


In [65]:
WeakUnf = SIS.Parameters(
    n=n*k,
    q=q,
    length_bound=zeta,
    m=n*(k+l),
    norm=oo,
    tag="WeakUnf"
)

print("*"+tag+": WeakUnf*")
r = SIS.estimate(WeakUnf, red_cost_model=ADPS16("classical"))

*Dilithium3: WeakUnf*
lattice  :: rop: ≈2^187.2, red: ≈2^187.2, sieve: ≈2^-332.2, β: 641, η: 641, ζ: 1, d: 2815, prob: 1, ↻: 1, tag: infinity


In [66]:
StrongUnf = SIS.Parameters(
    n=n*k,
    q=q,
    length_bound=zetap,
    m=n*(k+l),
    norm=oo,
    tag="StrongUnf"
)

print("*"+tag+": StrongUnf*")
r = SIS.estimate(StrongUnf, red_cost_model=ADPS16("classical"))

*Dilithium3: StrongUnf*
lattice  :: rop: ≈2^176.1, red: ≈2^176.1, sieve: ≈2^-332.2, β: 603, η: 603, ζ: 1, d: 2815, prob: 1, ↻: 1, tag: infinity


### Challenge Entropy

In [68]:
# Challenge Entropy
# Challenge has a Hamming weight of tau, and each coeff is 0 or +-1
# Thus, |challenge space| = (n choose tau)*2^tau
print("*"+tag+"*")
print("Challenge Entropy:", numerical_approx(log(binomial(n, tau))/log(2)+tau))

*Dilithium3*
Challenge Entropy: 225.345006273915


### Signature Forgery (lattice estimator)

In [52]:
schemes.Dilithium2_MSIS_WkUnf

SISParameters(n=1024, q=8380417, length_bound=350209, m=2304, norm=+Infinity, tag='Dilithium2_MSIS_WkUnf')

In [53]:
r = SIS.estimate(schemes.Dilithium2_MSIS_WkUnf)

lattice  :: rop: ≈2^152.2, red: ≈2^151.3, sieve: ≈2^151.1, β: 427, η: 433, ζ: 0, d: 2304, prob: 1, ↻: 1, tag: infinity


In [54]:
r = SIS.estimate(schemes.Dilithium2_MSIS_WkUnf, red_cost_model=ADPS16("classical"))

lattice  :: rop: ≈2^123.5, red: ≈2^123.5, sieve: ≈2^-332.2, β: 423, η: 423, ζ: 119, d: 2185, prob: 1, ↻: 1, tag: infinity


In [55]:
schemes.Dilithium2_MSIS_StrUnf 

SISParameters(n=1024, q=8380417, length_bound=380929, m=2304, norm=+Infinity, tag='Dilithium2_MSIS_StrUnf')

In [56]:
r = SIS.estimate(schemes.Dilithium2_MSIS_StrUnf)

lattice  :: rop: ≈2^150.8, red: ≈2^149.6, sieve: ≈2^149.9, β: 421, η: 429, ζ: 0, d: 2304, prob: 1, ↻: 1, tag: infinity


In [57]:
r = SIS.estimate(schemes.Dilithium2_MSIS_StrUnf, red_cost_model=ADPS16("classical"))

lattice  :: rop: ≈2^121.8, red: ≈2^121.8, sieve: ≈2^-332.2, β: 417, η: 417, ζ: 125, d: 2179, prob: 1, ↻: 1, tag: infinity


In [ ]:
r = SIS.estimate(schemes.Dilithium3_MSIS_WkUnf)

In [ ]:
r = SIS.estimate(schemes.Dilithium3_MSIS_WkUnf, red_cost_model=ADPS16("classical"))

In [ ]:
r = SIS.estimate(schemes.Dilithium3_MSIS_StrUnf)

In [ ]:
r = SIS.estimate(schemes.Dilithium3_MSIS_StrUnf, red_cost_model=ADPS16("classical"))

In [ ]:
r = SIS.estimate(schemes.Dilithium5_MSIS_WkUnf)

In [ ]:
r = SIS.estimate(schemes.Dilithium5_MSIS_WkUnf, red_cost_model=ADPS16("classical"))

In [ ]:
r = SIS.estimate(schemes.Dilithium5_MSIS_StrUnf)

In [ ]:
r = SIS.estimate(schemes.Dilithium5_MSIS_StrUnf, red_cost_model=ADPS16("classical"))